<a href="https://colab.research.google.com/github/chnyksl/basics/blob/master/nb/Phi_4-Conversational.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import json
import random

# Örnek veri (bunu rastgele üreteceğiz)
yorumlar = [
    "Ürün beklediğimden daha kaliteli çıktı, tavsiye ederim.",
    "Kargolama çok yavaştı, bir hafta sonra elime geçti.",
    "Fiyat performans açısından gayet başarılı.",
    "Paketleme özenliydi, ürün sağlam ulaştı.",
    "Renk görseldekinden biraz farklı geldi ama idare eder.",
    "Telefonun pili çabuk bitiyor, memnun kalmadım.",
    "Kumaşı güzel ama beden bir tık küçük geldi.",
    "Satıcı hızlı gönderim yaptı, teşekkür ederim.",
    "Kulaklık ses kalitesi çok iyi, dış sesleri güzel engelliyor.",
    "Ürün açıklamalardaki gibi değil, iade ettim."
]

# Örnek özetler (modelin öğreneceği hedef cümleler)
ozetler = [
    "Kaliteli ve tavsiye edilir ürün.",
    "Kargo gecikti.",
    "Fiyatına göre iyi performans.",
    "Paketleme sağlamdı.",
    "Renk farklı ama kabul edilebilir.",
    "Pil ömrü kısa.",
    "Beden küçük geldi.",
    "Hızlı gönderim için teşekkürler.",
    "Ses kalitesi mükemmel.",
    "Ürün beklentiyi karşılamadı."
]

# 500 örnek oluştur
dataset = []
for i in range(500):
    yorum = random.choice(yorumlar)
    ozet = random.choice(ozetler)
    example = {
        "instruction": "Ürünü 1 cümlede özetle",
        "input": yorum,
        "output": ozet
    }
    dataset.append(example)

# JSONL formatında kaydet
with open("review_dataset.jsonl", "w", encoding="utf-8") as f:
    for item in dataset:
        f.write(json.dumps(item, ensure_ascii=False) + "\n")

print("✅ 500 satırlık veri seti oluşturuldu: review_dataset.jsonl")

✅ 500 satırlık veri seti oluşturuldu: review_dataset.jsonl


In [2]:
import os
os.path.abspath("review_dataset.jsonl")

'/content/review_dataset.jsonl'

In [3]:
!pip install torch transformers datasets peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.8 MB/s eta 0:00:00


In [4]:
!python -m pip install torch transformers datasets peft accelerate bitsandbytes

In [5]:
from datasets import load_dataset
import json

# review_dataset.jsonl dosyanın yolunu yaz:
dataset_path = r"review_dataset.jsonl"  # <-- kendi dosya yolunu yaz

# JSONL dosyasını Dataset olarak yükle
dataset = load_dataset("json", data_files=dataset_path, split="train")

print(dataset[0])
print("Toplam örnek sayısı:", len(dataset))

Generating train split: 0 examples [00:00, ? examples/s]

{'instruction': 'Ürünü 1 cümlede özetle', 'input': 'Ürün açıklamalardaki gibi değil, iade ettim.', 'output': 'Ses kalitesi mükemmel.'}
Toplam örnek sayısı: 500


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

MODEL = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(MODEL, use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL)

# kısa kontrol
print("Tokenizer ve model yüklendi:", MODEL)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Tokenizer ve model yüklendi: google/flan-t5-small


In [8]:
from datasets import load_dataset
DATA_PATH = r"review_dataset.jsonl"  # kendi yolun

ds = load_dataset("json", data_files={"train":DATA_PATH})["train"]
print("Toplam örnek:", len(ds))
print("İlk örnek:", ds[0])

Toplam örnek: 500
İlk örnek: {'instruction': 'Ürünü 1 cümlede özetle', 'input': 'Ürün açıklamalardaki gibi değil, iade ettim.', 'output': 'Ses kalitesi mükemmel.'}


In [9]:
max_input_length = 256
max_target_length = 64

def preprocess_fn(example):
    # instruction + input -> prompt (gerekirse uyarlayabilirsin)
    prompt = f"{example.get('instruction','Özetle')}\n\n{example['input']}"
    model_inputs = tokenizer(prompt, max_length=max_input_length, truncation=True, padding="max_length")
    labels = tokenizer(example["output"], max_length=max_target_length, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tok_ds = ds.map(preprocess_fn, remove_columns=ds.column_names)
tok_ds = tok_ds.with_format("torch")
print("Hazır veri örneği:", tok_ds[0])

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

Hazır veri örneği: {'input_ids': tensor([    3, 14858,  4100,    29,  1272,   209,     3,    75, 21518,  1361,
           15,     3,  1872,   776,    17,   109,     3, 14858,  4100,    29,
            3,     9,  8970,     2,  8142,  1982,   986, 11259, 31257,    23,
           20,     2,   173,     6,     3,    23,     9,   221,     3, 10652,
           51,     5,     1,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,   

In [10]:
# Eğitime başla — CPU için hafif ayarlarla
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer, DataCollatorForSeq2Seq
import os

output_dir = "./ft-flan-t5-small"
os.makedirs(output_dir, exist_ok=True)

training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=4,    # CPU için 2-4 önerilir
    gradient_accumulation_steps=1,
    num_train_epochs=1,              # hızlı test için 1; sonra 2-3 yapabilirsin
    learning_rate=5e-5,
    logging_steps=50,
    save_strategy="epoch",
    fp16=False,                      # CPU -> False
    remove_unused_columns=False,
    predict_with_generate=True,
    report_to="none"
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tok_ds,
    tokenizer=tokenizer,
    data_collator=data_collator
)

print("Eğitim başlıyor — CPU olduğu için yavaş olacaktır. (İptal için Stop/Interrupt kullan.)")
trainer.train()
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)
print("Eğitim tamamlandı ve kaydedildi:", output_dir)


/tmp/ipython-input-1327928182.py:24: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Eğitim başlıyor — CPU olduğu için yavaş olacaktır. (İptal için Stop/Interrupt kullan.)


/usr/local/lib/python3.12/dist-packages/transformers/data/data_collator.py:740: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /pytorch/torch/csrc/utils/tensor_new.cpp:253.)
  batch["labels"] = torch.tensor(batch["labels"], dtype=torch.int64)


Step,Training Loss
50,21.098500
100,9.164300


Eğitim tamamlandı ve kaydedildi: ./ft-flan-t5-small


In [11]:
from transformers import pipeline, AutoModelForSeq2SeqLM

prompts = [
    "Ürünü 1 cümlede özetle: Bu telefonun bataryası 2 gün gidiyor ama kamerada sorun var.",
    "Ürünü 1 cümlede özetle: Yazıcı çok sessiz ama kağıt sıkıştırıyor.",
    "Ürünü 1 cümlede özetle: Ürünün rengi farklı geldi, iade ettim.",
    "Ürünü 1 cümlede özetle: Çok hafif ama bağlantı kopuyor.",
    "Ürünü 1 cümlede özetle: Fiyatına göre kalite çok iyi."
]

# Base
base_model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small")
base_pipe = pipeline("text2text-generation", model=base_model, tokenizer=tokenizer, device=-1)

# Fine-tuned
ft_model = AutoModelForSeq2SeqLM.from_pretrained(output_dir)
ft_pipe = pipeline("text2text-generation", model=ft_model, tokenizer=tokenizer, device=-1)

with open("results.txt", "w", encoding="utf-8") as f:
    f.write("=== BASE MODEL OUTPUTS ===\n\n")
    for p in prompts:
        out = base_pipe(p, max_length=64)[0]["generated_text"]
        f.write("PROMPT:\n"+p+"\nBASE:\n"+out+"\n\n")
    f.write("\n=== FINE-TUNED MODEL OUTPUTS ===\n\n")
    for p in prompts:
        out = ft_pipe(p, max_length=64)[0]["generated_text"]
        f.write("PROMPT:\n"+p+"\nFT:\n"+out+"\n\n")

print("Karşılaştırma kaydedildi: results.txt")
print(open("results.txt","r",encoding="utf-8").read())

Device set to use cpu
Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=256) and `max_length`(=64) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformer

Karşılaştırma kaydedildi: results.txt
=== BASE MODEL OUTPUTS ===

PROMPT:
Ürünü 1 cümlede özetle: Bu telefonun bataryası 2 gün gidiyor ama kamerada sorun var.
BASE:
Ülke 1: Bu telefonu telekomunikas 2 gidiyor olmak üzere gidiyor.

PROMPT:
Ürünü 1 cümlede özetle: Yazıcı çok sessiz ama kağıt sıkıştırıyor.
BASE:
Ülke 1 yaşndaki açklamada: Yazc çocuklarnn çocuklarna kabul edildiini söylüyor.

PROMPT:
Ürünü 1 cümlede özetle: Ürünün rengi farklı geldi, iade ettim.
BASE:
lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1: lensk 1:

PROMPT:
Ürünü 1 cümlede özetle: Çok hafif ama bağlantı kopuyor.
BASE:
lensk 1: ok hafif ama balant.

PROMPT:
Ürünü 1 cümlede özetle: Fiyatına göre kalite çok iyi.
BASE:
Ürünü 1: Fiyatna göre kalite kalite.


=== FINE-TUNED MODEL OUTPUTS ===

